In [1]:
#Create the initial concatenated dataframe from raw data

from pyspark.sql.functions import lit

# Load in one of the tables
df1 = spark.sql("select * from default.video_games_5")
df1 = df1.withColumn('category', lit("video_games"))

df2 = spark.sql("select * from default.home_and_kitchen_5_small")
df2 = df2.withColumn('category', lit("home_and_kitchen"))

df3 = spark.sql("select * from default.books_5_small")
df3 = df3.withColumn('category', lit("books"))

df = df1.union(df2).union(df3)

test_df = spark.sql("select * from default.reviews_kaggle")

# Take a sample (useful for code development purposes)
#df = df.sample(False, 0.15, seed=0)

df = df.cache()
test_df = test_df.cache()

print((df.count(), len(df.columns)))

(3487331, 13)

In [2]:
#Non-Textual/Creation Pre-processing

#Conversion of string date (reviewTime) to date date; drop reviewTime
from pyspark.sql.functions import to_date

df = df.withColumn('Review_Date', to_date('reviewTime', 'MM dd, yyyy'))
df = df.drop('reviewTime')

display(df)        

#Repeat for Kaggle test dataset
test_df = test_df.withColumn('Review_Date', to_date('reviewTime', 'MM dd, yyyy'))
test_df = test_df.drop('reviewTime')

#we will drop vote, reviewID, unixReviewTime

reviewID,overall,vote,verified,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,label,category,Review_Date
0,5.0,0,true,A1HP7NVNPFMA4N,0700026657,Ambrosia075,"This game is a bit hard to get the hang of, but when you do it's great.",but when you do it's great.,1445040000,0,video_games,2015-10-17
1,4.0,0,false,A1JGAP0185YJI6,0700026657,travis,"I played it a while but it was alright. The steam was a bit of trouble. The more they move these game to steam the more of a hard time I have activating and playing a game. But in spite of that it was fun, I liked it. Now I am looking forward to anno 2205 I really want to play my way to the moon.","But in spite of that it was fun, I liked it",1437955200,0,video_games,2015-07-27
2,3.0,0,true,A1YJWEXHQBWK2B,0700026657,Vincent G. Mezera,ok game.,Three Stars,1424649600,0,video_games,2015-02-23
3,2.0,0,true,A2204E1TH211HT,0700026657,Grandma KR,"found the game a bit too complicated, not what I expected after having played 1602, 1503, and 1701",Two Stars,1424390400,0,video_games,2015-02-20
4,5.0,0,true,A2RF5B5H74JLPE,0700026657,jon,"great game, I love it and have played it since its arrived",love this game,1419465600,0,video_games,2014-12-25
5,4.0,0,true,A11V6ZJ2FVQY1D,0700026657,IBRAHIM ALBADI,i liked a lot some time that i haven't play a wonderfull game very simply and funny game verry good game.,Anno 2070,1415836800,0,video_games,2014-11-13
6,1.0,0,false,A1KXJ1ELZIU05C,0700026657,Creation27,"I'm an avid gamer, but Anno 2070 is an INSULT to gaming. It is so buggy and half-finished that the first campaign doesn't even work properly and the DRM is INCREDIBLY frustrating to deal with. Once you manage to work your way past the massive amounts of bugs and get through the DRM, HOURS later you finally figure out that the game has no real tutorial, so you stuck just clicking around randomly. Sad, sad, sad, example of a game that could have been great but FTW.",Avoid This Game - Filled with Bugs,1406937600,0,video_games,2014-08-02
7,5.0,0,true,A1WK5I4874S3O2,0700026657,WhiteSkull,"I bought this game thinking it would be pretty cool and that i might play it for a week or two and be done. Boy was I wrong! From the moment I finally got the gamed Fired up (the other commentors on this are right, it takes forever and u are forced to create an account) I watched as it booted up I could tell right off the bat that ALOT of thought went into making this game. If you have ever played Sim city, then this game is a must try as you will easily navigate thru it and its multi layers. I have been playing htis now for a month straight, and I am STILL discovering layers of complexity in the game. There are a few things in the game that could used tweaked, but all in all this is a 5 star game.",A very good game balance of skill with depth of choices,1393804800,0,video_games,2014-03-03
8,5.0,0,true,AV969NA4CBP10,0700026657,Travis B. Moore,I have played the old anno 1701 AND 1503. this game looks great but is more complex than the previous versions of the game. I found a lot of things lacking such as the sources of power and an inability to store energy with batteries or regenertive fuel cells as buildings in the game need power. Trade is about the same. My main beef with this it requires an internet connection. Other than that it has wonderful artistry and graphics. It is the same as anno 1701 but set in a future world where global warmming as flood the land and resource scarcity has sent human kind to look to the deep ocean for valuable minerals. I recoment the deep ocean expansion or complete if you get this. I found the ai instructor a little corny but other than that the game has some real polish. I wrote my 2 cents worth on suggestions on anno 2070 wiki and you can read 3 pages on that for game ideas I had.,Anno 2070 more like anno 1701,1392940800,0,video_games,2014-02-21
9,4.0,0,true,A1EO9BFUHTGWKZ,0700026657,johnnyz3,"I liked it and had fun with it, played for a while and got my money's worth. You can 

In [3]:
##############################################Feature Engineering##################################################################
###################################################################################################################################

#Dealing with customers with multiple reviews: review number (Partition dataset by customerID and review date (asc) to get the review # per customer)

#aggregate on review # (by product category, overall (rating)) to look at review habits over time (i.e did helpfulness increase over time? did length/other text features change over time)

from pyspark.sql.window import *
from pyspark.sql.functions import row_number
import pyspark.sql.functions as F
from textblob import TextBlob

#creation and addition of review_number feature (the nth review of a particular reviewer/customer)
#df = df.withColumn("review_number", row_number().over(Window.partitionBy("reviewerID").orderBy("unixReviewTime"))).sort("reviewerID","review_number")

#taking this a step further, let's look at the review_number within a category
# df = df.withColumn("review_number_within_category", row_number().over(Window.partitionBy("reviewerID","category").orderBy("reviewDate"))).sort("reviewerID","category","review_number_within_category")

# display(df)

#another numbered feature - the review number for a product; let's comment these for now as we have this feature with both test+train datasets already
#df = df.withColumn("product_review_number", row_number().over(Window.partitionBy("asin").orderBy("unixReviewTime")))

##########################Date features: day of week and month##################
# from pyspark.sql.functions import dayofweek, date_format, month
# df = df.withColumn("Review_DoW",dayofweek('Review_Date'))
# df = df.withColumn("Review_Month",month('Review_Date'))

#display(df)

##########################Text Analyics Features################################
from pyspark.sql.functions import size, split, col
#review word count
df = df.withColumn('Review_Wordcount', size(split(col('reviewText'), ' ')))

#review character length
df = df.withColumn('Review_Length', F.length(col('reviewText')))

#review sentiment scoring (via textblob - do both on reviewText and summary?
sentiment = udf(lambda x: TextBlob(x).sentiment[0])
spark.udf.register("Sentiment", sentiment)
df = df.withColumn("Sentiment",sentiment("reviewText").cast("double"))

#review readability score (flesch_reading_ease via textstat library)

display(df)


##########################Customer-Level Features################################






reviewID,overall,vote,verified,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,label,category,Review_Date,Review_Wordcount,Review_Length,Sentiment
0,5.0,0,true,A1HP7NVNPFMA4N,0700026657,Ambrosia075,"This game is a bit hard to get the hang of, but when you do it's great.",but when you do it's great.,1445040000,0,video_games,2015-10-17,17,71,0.03611111111111113
1,4.0,0,false,A1JGAP0185YJI6,0700026657,travis,"I played it a while but it was alright. The steam was a bit of trouble. The more they move these game to steam the more of a hard time I have activating and playing a game. But in spite of that it was fun, I liked it. Now I am looking forward to anno 2205 I really want to play my way to the moon.","But in spite of that it was fun, I liked it",1437955200,0,video_games,2015-07-27,66,297,0.08981481481481479
2,3.0,0,true,A1YJWEXHQBWK2B,0700026657,Vincent G. Mezera,ok game.,Three Stars,1424649600,0,video_games,2015-02-23,2,8,0.04999999999999999
3,2.0,0,true,A2204E1TH211HT,0700026657,Grandma KR,"found the game a bit too complicated, not what I expected after having played 1602, 1503, and 1701",Two Stars,1424390400,0,video_games,2015-02-20,18,98,-0.3333333333333333
4,5.0,0,true,A2RF5B5H74JLPE,0700026657,jon,"great game, I love it and have played it since its arrived",love this game,1419465600,0,video_games,2014-12-25,12,58,0.3
5,4.0,0,true,A11V6ZJ2FVQY1D,0700026657,IBRAHIM ALBADI,i liked a lot some time that i haven't play a wonderfull game very simply and funny game verry good game.,Anno 2070,1415836800,0,video_games,2014-11-13,21,105,0.04999999999999998
6,1.0,0,false,A1KXJ1ELZIU05C,0700026657,Creation27,"I'm an avid gamer, but Anno 2070 is an INSULT to gaming. It is so buggy and half-finished that the first campaign doesn't even work properly and the DRM is INCREDIBLY frustrating to deal with. Once you manage to work your way past the massive amounts of bugs and get through the DRM, HOURS later you finally figure out that the game has no real tutorial, so you stuck just clicking around randomly. Sad, sad, sad, example of a game that could have been great but FTW.",Avoid This Game - Filled with Bugs,1406937600,0,video_games,2014-08-02,88,468,-0.11666666666666665
7,5.0,0,true,A1WK5I4874S3O2,0700026657,WhiteSkull,"I bought this game thinking it would be pretty cool and that i might play it for a week or two and be done. Boy was I wrong! From the moment I finally got the gamed Fired up (the other commentors on this are right, it takes forever and u are forced to create an account) I watched as it booted up I could tell right off the bat that ALOT of thought went into making this game. If you have ever played Sim city, then this game is a must try as you will easily navigate thru it and its multi layers. I have been playing htis now for a month straight, and I am STILL discovering layers of complexity in the game. There are a few things in the game that could used tweaked, but all in all this is a 5 star game.",A very good game balance of skill with depth of choices,1393804800,0,video_games,2014-03-03,148,709,-0.10854341736694678
8,5.0,0,true,AV969NA4CBP10,0700026657,Travis B. Moore,I have played the old anno 1701 AND 1503. this game looks great but is more complex than the previous versions of the game. I found a lot of things lacking such as the sources of power and an inability to store energy with batteries or regenertive fuel cells as buildings in the game need power. Trade is about the same. My main beef with this it requires an internet connection. Other than that it has wonderful artistry and graphics. It is the same as anno 1701 but set in a future world where global warmming as flood the land and resource scarcity has sent human kind to look to the deep ocean for valuable minerals. I recoment the deep ocean expansion or complete if you get this. I found the ai instructor a little corny but other than that the game has some real polish. I wrote my 2 cents worth on suggestions on anno 2070 wiki and you can read 

In [8]:
#sorry for screaming this text out - here's a really well shaped potato: O; ok i'll be nice - 2 potatoes: OO

In [9]:
#adding class weights to counter the imbalanced number of helpful, unhelpful reviews

import pyspark.sql.functions as f
from pyspark.sql.types import FloatType

# Re-balancing (weighting) of records to be used in the logistic loss objective function
numPositives = df.filter(df["label"] == 1).count()
datasetSize = df.count()
balancingRatio = (datasetSize - numPositives) / datasetSize
print("numPositives   = {}".format(numPositives))
print("datasetSize    = {}".format(datasetSize))
print("balancingRatio = {}".format(balancingRatio))

def calculateWeights(d, multiplier = 1):
    if d == 1.0:
      return multiplier * balancingRatio
    else:
      return (1.0 - balancingRatio)
    
udfcalculateWeights = f.udf(calculateWeights, FloatType())
    
df = df.withColumn("classWeightCol", udfcalculateWeights(df["label"]))

display(df)

reviewID,overall,vote,verified,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,label,category,Review_Date,Review_Wordcount,Review_Length,Sentiment,classWeightCol
0,5.0,0,true,A1HP7NVNPFMA4N,0700026657,Ambrosia075,"This game is a bit hard to get the hang of, but when you do it's great.",but when you do it's great.,1445040000,0,video_games,2015-10-17,17,71,0.03611111111111113,0.1795545
1,4.0,0,false,A1JGAP0185YJI6,0700026657,travis,"I played it a while but it was alright. The steam was a bit of trouble. The more they move these game to steam the more of a hard time I have activating and playing a game. But in spite of that it was fun, I liked it. Now I am looking forward to anno 2205 I really want to play my way to the moon.","But in spite of that it was fun, I liked it",1437955200,0,video_games,2015-07-27,66,297,0.08981481481481479,0.1795545
2,3.0,0,true,A1YJWEXHQBWK2B,0700026657,Vincent G. Mezera,ok game.,Three Stars,1424649600,0,video_games,2015-02-23,2,8,0.04999999999999999,0.1795545
3,2.0,0,true,A2204E1TH211HT,0700026657,Grandma KR,"found the game a bit too complicated, not what I expected after having played 1602, 1503, and 1701",Two Stars,1424390400,0,video_games,2015-02-20,18,98,-0.3333333333333333,0.1795545
4,5.0,0,true,A2RF5B5H74JLPE,0700026657,jon,"great game, I love it and have played it since its arrived",love this game,1419465600,0,video_games,2014-12-25,12,58,0.3,0.1795545
5,4.0,0,true,A11V6ZJ2FVQY1D,0700026657,IBRAHIM ALBADI,i liked a lot some time that i haven't play a wonderfull game very simply and funny game verry good game.,Anno 2070,1415836800,0,video_games,2014-11-13,21,105,0.04999999999999998,0.1795545
6,1.0,0,false,A1KXJ1ELZIU05C,0700026657,Creation27,"I'm an avid gamer, but Anno 2070 is an INSULT to gaming. It is so buggy and half-finished that the first campaign doesn't even work properly and the DRM is INCREDIBLY frustrating to deal with. Once you manage to work your way past the massive amounts of bugs and get through the DRM, HOURS later you finally figure out that the game has no real tutorial, so you stuck just clicking around randomly. Sad, sad, sad, example of a game that could have been great but FTW.",Avoid This Game - Filled with Bugs,1406937600,0,video_games,2014-08-02,88,468,-0.11666666666666665,0.1795545
7,5.0,0,true,A1WK5I4874S3O2,0700026657,WhiteSkull,"I bought this game thinking it would be pretty cool and that i might play it for a week or two and be done. Boy was I wrong! From the moment I finally got the gamed Fired up (the other commentors on this are right, it takes forever and u are forced to create an account) I watched as it booted up I could tell right off the bat that ALOT of thought went into making this game. If you have ever played Sim city, then this game is a must try as you will easily navigate thru it and its multi layers. I have been playing htis now for a month straight, and I am STILL discovering layers of complexity in the game. There are a few things in the game that could used tweaked, but all in all this is a 5 star game.",A very good game balance of skill with depth of choices,1393804800,0,video_games,2014-03-03,148,709,-0.10854341736694678,0.1795545
8,5.0,0,true,AV969NA4CBP10,0700026657,Travis B. Moore,I have played the old anno 1701 AND 1503. this game looks great but is more complex than the previous versions of the game. I found a lot of things lacking such as the sources of power and an inability to store energy with batteries or regenertive fuel cells as buildings in the game need power. Trade is about the same. My main beef with this it requires an internet connection. Other than that it has wonderful artistry and graphics. It is the same as anno 1701 but set in a future world where global warmming as flood the land and resource scarcity has sent human kind to look to the deep ocean for valuable minerals. I recoment the deep ocean expansion or complete if you get this. I found the ai instructor a little corny but other than that the game ha

In [10]:
#finalize df before modeling attempt
mini_df = df['reviewID', 'reviewerID', 'Review_Date', 'asin','verified']

#bring in kaggle test set
kaggle_mini_df = test_df['reviewID', 'reviewerID', 'Review_Date', 'asin','verified']

#union the two
mini_union_df = mini_df.union(kaggle_mini_df)

#feature engineer user review number
mini_union_df = mini_union_df.withColumn("review_number_new", row_number().over(Window.partitionBy("reviewerID").orderBy("Review_Date"))).sort("reviewerID","review_number_new")

#feature engineer product review number
mini_union_df = mini_union_df.withColumn("product_review_number_new", row_number().over(Window.partitionBy("asin").orderBy("Review_Date")))

#feature engineer Recency metric (days_since_last_review)
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.functions import datediff
from pyspark.sql.window import Window
import sys


my_window = Window.partitionBy("reviewerID").orderBy("review_number_new")#.rowsBetween(-sys.maxsize,0)

mini_union_df = mini_union_df.withColumn("prev_date", F.lag(mini_union_df.Review_Date).over(my_window))
mini_union_df = mini_union_df.withColumn("days_since_last_review", F.when(F.isnull(datediff(mini_union_df.Review_Date, mini_union_df.prev_date)), 0)
                              .otherwise(datediff(mini_union_df.Review_Date, mini_union_df.prev_date)))

#feature engineer Monetary metric (count_verified)
my_window2 = Window.partitionBy("reviewerID").orderBy("review_number_new").rowsBetween(-sys.maxsize,0)

mini_union_df = mini_union_df.withColumn("v_value", F.when(F.col('verified') == 'true', 1).otherwise(0))

mini_union_df = mini_union_df.withColumn("count_verified", F.sum(mini_union_df['v_value']).over(my_window2))

display(mini_union_df)

reviewID,reviewerID,Review_Date,asin,verified,review_number_new,product_review_number_new,prev_date,days_since_last_review,v_value,count_verified
1489616,A0232309XTWAG6FEGW93,2014-12-02,B001FB6TSA,true,1,73,null,0,1,1
1501779,A0232309XTWAG6FEGW93,2014-12-02,B001G5ZZCQ,true,2,14,2014-12-02,0,1,2
8590118359,A02836981FYG9912C66F,2015-11-02,B0114I1Z50,false,1,5,null,0,0,0
8590102001,A02836981FYG9912C66F,2015-11-18,B00W8FYFBA,false,2,23,2015-11-02,16,0,0
8590107900,A02836981FYG9912C66F,2016-03-06,B00YQ2KCWO,false,3,306,2015-11-18,109,0,0
8590055103,A02836981FYG9912C66F,2016-07-07,B00JIJUB7G,true,4,9,2016-03-06,123,1,1
8590128560,A02836981FYG9912C66F,2016-09-05,B016KBVBCS,true,5,163,2016-07-07,60,1,2
8590145137,A02836981FYG9912C66F,2016-11-30,B01GD490UM,true,6,42,2016-09-05,86,1,3
8590147486,A02836981FYG9912C66F,2017-08-28,B01GVE4ZXQ,true,7,23,2016-11-30,271,1,4
1476105,A100CUKWB6DEJR,2015-11-23,B001EYFQ28,true,1,927,null,0,1,1


In [11]:
#checking if any reviewers are present in both the train/test dataset and the kaggle prediction 
df_mini_check = mini_df.join(kaggle_mini_df, mini_df.reviewerID == kaggle_mini_df.reviewerID)
display(df_mini_check)
#observations: we confirm that there are common cases

reviewID,reviewerID,Review_Date,asin,verified,reviewID,reviewerID,Review_Date,asin,verified
54846,A108E2Q3U3CDFL,2018-03-15,B00004W4UQ,true,67059980,A108E2Q3U3CDFL,2017-03-27,142620003X,true
145079,A108E2Q3U3CDFL,2018-03-15,B00008UA3Y,true,67059980,A108E2Q3U3CDFL,2017-03-27,142620003X,true
571881,A108E2Q3U3CDFL,2014-08-25,B000HJBH24,true,67059980,A108E2Q3U3CDFL,2017-03-27,142620003X,true
1085621,A108E2Q3U3CDFL,2018-03-15,B0017XHSC2,true,67059980,A108E2Q3U3CDFL,2017-03-27,142620003X,true
1314441,A108E2Q3U3CDFL,2018-03-15,B0017XHSC2,true,67059980,A108E2Q3U3CDFL,2017-03-27,142620003X,true
101335,A10IIGVW3NGBH9,2016-06-04,B000063SKE,true,67134336,A10IIGVW3NGBH9,2016-06-04,B01BFQK54S,true
247590,A10IIGVW3NGBH9,2016-04-10,B0000X1146,true,67134336,A10IIGVW3NGBH9,2016-06-04,B01BFQK54S,true
1005992,A10IIGVW3NGBH9,2016-04-10,B0014E0UFQ,true,67134336,A10IIGVW3NGBH9,2016-06-04,B01BFQK54S,true
1234812,A10IIGVW3NGBH9,2016-04-10,B0014E0UFQ,true,67134336,A10IIGVW3NGBH9,2016-06-04,B01BFQK54S,true
29011,A10KHX41ONY4U1,2013-05-23,B00005QEFD,true,67204643,A10KHX41ONY4U1,2013-11-17,1479140201,true


In [12]:
#doris' kaggle record
display(test_df.filter(col('reviewerID') == 'A108E2Q3U3CDFL'))

reviewID,overall,verified,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,Review_Date
67059980,5,true,A108E2Q3U3CDFL,142620003X,Doris,"a lovely book. I bought it to use with a little girl, just to show her the lictures, since she is going to be a big sister soon.",a lovely book. I bought it to use with ...,1.4905728E9,2017-03-27


In [13]:
#check for doris
display(mini_union_df.filter(col('reviewerID') == 'A108E2Q3U3CDFL'))

reviewID,reviewerID,Review_Date,asin,verified,review_number_new,product_review_number_new,prev_date,days_since_last_review,v_value,count_verified
571881,A108E2Q3U3CDFL,2014-08-25,B000HJBH24,true,1,19,null,0,1,1
67059980,A108E2Q3U3CDFL,2017-03-27,142620003X,true,2,6,2014-08-25,945,1,2
54846,A108E2Q3U3CDFL,2018-03-15,B00004W4UQ,true,3,837,2017-03-27,353,1,3
145079,A108E2Q3U3CDFL,2018-03-15,B00008UA3Y,true,4,2429,2018-03-15,0,1,4
1085621,A108E2Q3U3CDFL,2018-03-15,B0017XHSC2,true,5,3225,2018-03-15,0,1,5
1314441,A108E2Q3U3CDFL,2018-03-15,B0017XHSC2,true,6,3226,2018-03-15,0,1,6


In [14]:
#reviewID is NOT UNIQUE in df!! 

check = df.groupBy("reviewID").count()
display(check)

#Each category has its own reviewID counter it seems, so we don't have to worry about duplicate reviews
display(df.filter(df.reviewID==725295))

reviewID,overall,vote,verified,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,label,category,Review_Date,Review_Wordcount,Review_Length,Sentiment,classWeightCol
725295,5.0,0,true,A11CSBZN1W50EU,B000NTNA0K,myaari,Adorable,Five Stars,1426118400,0,home_and_kitchen,2015-03-12,1,8,0.5,0.1795545
725295,5.0,0,true,A1M163XB9J5T6I,0060628723,Peppilin,This is a wonderful book to have. I use it as a resource for our Wednesday evening Vespers and often share its selections with the congregation to help give them a taste of various thinkers throughout the Church.,This is a wonderful book to have,1461628800,0,books,2016-04-26,38,212,0.5,0.1795545


In [15]:
#check df sizes pre-join
print((df.count(), len(df.columns)))
print((test_df.count(), len(test_df.columns)))

(3487331, 17)
(304984, 10)

In [16]:
# join updated user review count and product review count back into df and test_df i.e. original train/test df and kaggle prediction df

def join(df1, df2, cond, how='left'):
    df = df1.join(df2, cond, how=how)
    repeated_columns = [c for c in df1.columns if c in df2.columns]
    for col in repeated_columns:
        df = df.drop(df2[col])
    return df
  
df2 = join(df, mini_union_df, (df.reviewID == mini_union_df.reviewID) & (df.reviewerID == mini_union_df.reviewerID))

test_df2 = join(test_df, mini_union_df, (test_df.reviewID == mini_union_df.reviewID) & (test_df.reviewerID == mini_union_df.reviewerID))

print((df2.count(), len(df2.columns)))
print((test_df2.count(), len(test_df2.columns)))

(3487331, 23)
(304984, 16)

In [17]:
#examine our joined (and confirm doris)
display(df2.filter(col('reviewerID') == 'A108E2Q3U3CDFL'))
#observations: we indeed see that Doris records in training set only reflect reviewNumber 1,3-6 (reviewNumber 2 is in the Kaggle test set)

overall,vote,reviewerName,reviewText,summary,unixReviewTime,label,category,Review_Wordcount,Review_Length,Sentiment,classWeightCol,reviewID,reviewerID,Review_Date,asin,verified,review_number_new,product_review_number_new,prev_date,days_since_last_review,v_value,count_verified
4.0,0,Doris,One of the pans had dings on it when I unwrapped it. But the pans work well and when I burnt something on one of them it came right off. The pans are good quality and they weigh a good solid amount - instead of being flimsy and thin. The dings are a shame but I still like the quality of this product.,"solid and servicable, cleans easily",1521072000,0,home_and_kitchen,63,301,0.2571428571428571,0.1795545,145079,A108E2Q3U3CDFL,2018-03-15,B00008UA3Y,true,4,2429,2018-03-15,0,1,4
3.0,0,Doris,already reviewed this,"light weight product, only able to do a small amount of work",1521072000,0,home_and_kitchen,3,21,0.0,0.1795545,1314441,A108E2Q3U3CDFL,2018-03-15,B0017XHSC2,true,6,3226,2018-03-15,0,1,6
3.0,0,Doris,"It did melt on one side after about two months of use. But something caught fire in the microwave, so that is probably the reason why. It looks nice and was working well until the fire.","Looks nice, works reasonably well",1521072000,0,home_and_kitchen,36,185,0.6,0.1795545,54846,A108E2Q3U3CDFL,2018-03-15,B00004W4UQ,true,3,837,2017-03-27,353,1,3
4.0,0,Doris,"I bought these to use in making a craft for Christmas. They are sturdy and nice looking, and they worked well for my purposes. The holes were a little bit too generous, however, for use at table. These are better as salt shakers on the stove for cooking, than as salt shakers used during as meal for seasoning a plate of food.","sturdy, nice looking",1408924800,0,home_and_kitchen,63,327,0.30416666666666664,0.1795545,571881,A108E2Q3U3CDFL,2014-08-25,B000HJBH24,true,1,19,null,0,1,1
3.0,0,Doris,already reviewed this,"light weight product, only able to do a small amount of work",1521072000,0,home_and_kitchen,3,21,0.0,0.1795545,1085621,A108E2Q3U3CDFL,2018-03-15,B0017XHSC2,true,5,3225,2018-03-15,0,1,5


In [18]:
#finalizing df for model
df2 = df2.drop('vote', 'unixReviewTime', 'Review_DoW', 'Review_Month', 'reviewID', 'Review_Date', 'review_number', 'category')
#what do we do with classWeightCol...?

In [19]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer, VectorAssembler, IDF, NGram
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, NaiveBayes, MultilayerPerceptronClassifier, GBTClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
import xgboost as xgb

# We'll tokenize the text using a simple RegexTokenizer
tokenizer = RegexTokenizer(inputCol="reviewText", outputCol="words", pattern="\\W")

# Remove standard Stopwords
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered")

# Convert to lower case, remove punctuation, remove numbers?

# TODO: insert other cleaning steps here (and put into the pipeline, of course!)
# E.g., n-grams? document length?

#n-grams (let's try 2)
#ngrams = NGram(n=2, inputCol="filtered", outputCol="ngrams")

# Vectorize the sentences using simple BOW method. Other methods are possible:
# https://spark.apache.org/docs/2.2.0/ml-features.html#feature-extractors #what about vocab size 200?

tf = CountVectorizer(inputCol="filtered", outputCol="rawFeatures", vocabSize=2000, minTF=1, maxDF=0.40)

# Generate Inverse Document Frequency weighting
idf = IDF(inputCol="rawFeatures", outputCol="idfFeatures", minDocFreq=100)

# Combine all features into one final "features" column
assembler = VectorAssembler(inputCols=["verified", "overall", "review_number_new", "product_review_number_new", "days_since_last_review", "count_verified", "Review_Wordcount", "Review_Length", "Sentiment", "idfFeatures"], outputCol="features")

#drop Sentiment for NB classifier as it doesn't accept negative feature values
# assembler = VectorAssembler(inputCols=["verified", "overall", "review_number_new", "product_review_number_new", "Review_Wordcount", "Review_Length", "idfFeatures"], outputCol="features")

# Machine Learning Algorithm
#ml_alg  = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.0, weightCol="classWeightCol")
#ml_alg  = LogisticRegression(maxIter=10, weightCol="classWeightCol")
#ml_alg  = NaiveBayes(smoothing=1.0, modelType="multinomial", weightCol="classWeightCol")
#ml_alg  = RandomForestClassifier(numTrees=100, featureSubsetStrategy="auto", impurity='gini', maxDepth=4, maxBins=32)
#ml_alg = MultilayerPerceptronClassifier(maxIter=100, layers=[len(assembler.getInputCols()), 5, 5, 2], blockSize=128, seed=47)
ml_alg = GBTClassifier(maxDepth=5, maxBins=20, subsamplingRate=, stepSize=0.01)



pipeline = Pipeline(stages=[tokenizer, stopwordsRemover, tf, idf, assembler, ml_alg])

# paramGrid = ParamGridBuilder() \
#     .addGrid(ml_alg.regParam, [0.0, 0.19]) \
#     .addGrid(ml_alg.elasticNetParam, [0.0, 0.5]) \
#     .build()

# evaluator = BinaryClassificationEvaluator(metricName="areaUnderROC")

# crossval = CrossValidator(estimator=pipeline,
#                           estimatorParamMaps=paramGrid,
#                           evaluator=evaluator,
#                           numFolds=3)  # using 4 folds as it is suggested to use 3+ folds in practice

File "<command-3883965750813997>" , line 42 
 ml_alg = GBTClassifier(maxDepth=5, maxBins=20, subsamplingRate=, stepSize=0.01) 
 ^
 SyntaxError : invalid syntax

--------------------------------------------------------------------------- 
 ModuleNotFoundError Traceback (most recent call last)
 <command-529355328023468> in <module> () 
 ----> 1 import asdfasdf as xgb

 ModuleNotFoundError : No module named 'asdfasdf'

In [21]:
#
df2 = df2.cache()

# #manual undersampling for algos without weightCol param
# major_df2 = df2.filter(df2.label == 0)
# minor_df2 = df2.filter(df2.label == 1)
# ratio = int(major_df2.count()/minor_df2.count())
# print("ratio: {}".format(ratio)) 

# sampled_majority_df = major_df2.sample(False, 1/ratio)
# combined_df_2 = sampled_majority_df.unionAll(minor_df2)

(trainingData, testData) = df2.randomSplit([0.67, 0.33], seed = 47) #Use this line for algo's with built in weightCols
#(trainingData, testData) = combined_df_2.randomSplit([0.67, 0.33], seed = 47) #NOTE CHANGE df2 back to df;  set seed for reproducibility
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count:     " + str(testData.count()))

trainingData = trainingData.cache()
testData = testData.cache()

Training Dataset Count: 2336018
Test Dataset Count: 1151313

Out[25]: 419303

Out[21]: 2861165

In [24]:
#pipelineFit = crossval.fit(trainingData)
pipelineFit = pipeline.fit(trainingData)

In [25]:
#preview training df
#pipeline2 = Pipeline(stages=[tokenizer, stopwordsRemover, tf, idf, assembler])

#pipelineFit2 = pipeline2.fit(trainingData)

In [26]:
predictions =  pipelineFit.transform(testData)
predictions.groupBy("prediction").count().show()

+----------+------+
prediction| count|
+----------+------+
 0.0|224073|
 1.0|218639|
+----------+------+

In [27]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

acc_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
pre_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
rec_evaluator = MulticlassClassificationEvaluator(metricName="weightedRecall")
pr_evaluator  = BinaryClassificationEvaluator(metricName="areaUnderPR")
auc_evaluator = BinaryClassificationEvaluator(metricName="areaUnderROC")

print("Test Accuracy       = %g" % (acc_evaluator.evaluate(predictions)))
print("Test Precision      = %g" % (pre_evaluator.evaluate(predictions)))
print("Test Recall         = %g" % (rec_evaluator.evaluate(predictions)))
print("Test areaUnderPR    = %g" % (pr_evaluator.evaluate(predictions)))
print("Test areaUnderROC   = %g" % (auc_evaluator.evaluate(predictions)))

Test Accuracy = 0.788007
Test Precision = 0.78956
Test Recall = 0.788007
Test areaUnderPR = 0.83972
Test areaUnderROC = 0.869424

In [28]:
# Load in the tables
#loading test table in cmd 1
test_df2.show(5)
print((test_df2.count(), len(test_df2.columns)))
test_df2.describe().show()

In [29]:
# #Non-Textual/Creation Pre-processing

# #Conversion of string date (reviewTime) to date date; drop reviewTime
# from pyspark.sql.functions import to_date

# test_df = test_df.withColumn('Review_Date', to_date('reviewTime', 'MM dd, yyyy'))
# test_df = test_df.drop('reviewTime')

# display(test_df)        

# #we will drop vote, reviewID, unixReviewTime

In [30]:
#adding features into test_df2

##############################################Feature Engineering##################################################################
###################################################################################################################################

#Dealing with customers with multiple reviews: review number (Partition dataset by customerID and review date (asc) to get the review # per customer)

#aggregate on review # (by product category, overall (rating)) to look at review habits over time (i.e did helpfulness increase over time? did length/other text features change over time)

from pyspark.sql.window import *
from pyspark.sql.functions import row_number
import pyspark.sql.functions as F

#creation and addition of review_number feature (the nth review of a particular reviewer/customer)
#test_df2 = test_df2.withColumn("review_number", row_number().over(Window.partitionBy("reviewerID").orderBy("unixReviewTime"))).sort("reviewerID","review_number")

#taking this a step further, let's look at the review_number within a category
#df = df.withColumn("review_number_within_category", row_number().over(Window.partitionBy("reviewerID","category").orderBy("reviewDate"))).sort("reviewerID","category","review_number_within_category")

#another numbered feature - the review number for a product
#test_df2 = test_df2.withColumn("product_review_number", row_number().over(Window.partitionBy("asin").orderBy("unixReviewTime")))

##########################Date features: day of week and month##################
from pyspark.sql.functions import dayofweek, date_format, month
test_df2 = test_df2.withColumn("Review_DoW",dayofweek('Review_Date'))
test_df2 = test_df2.withColumn("Review_Month",month('Review_Date'))

##########################Text Analyics Features################################
from pyspark.sql.functions import size, split, col
#review word count
test_df2 = test_df2.withColumn('Review_Wordcount', size(split(col('reviewText'), ' ')))

#review character length
test_df2 = test_df2.withColumn('Review_Length', F.length(col('reviewText')))

#review sentiment scoring (via textblob - do both on reviewText and summary?)
sentiment = udf(lambda x: TextBlob(x).sentiment[0])
spark.udf.register("Sentiment", sentiment)
test_df2 = test_df2.withColumn("Sentiment",sentiment("reviewText").cast("double"))

#review readability score (flesch_reading_ease via textstat library)

display(test_df2)

reviewID overall verified reviewerID asin reviewerName reviewText summary unixReviewTime Review_Date review_number_new product_review_number_new prev_date days_since_last_review v_value count_verified Review_DoW Review_Month Review_Wordcount Review_Length Sentiment 67000781 5 false AWG41315YSXQM B00YTNMHEQ Christine Frances This book was exciting, to say the least. Very easy to picture all the action, as the author writes the script he wants you to follow! He did a good job, in my opinion. Wow, wow, wow! 1.5109632E9 2017-11-18 1 1 null 0 0 0 7 11 34 168 0.2688888888888889 67001078 5 true A2FXMDRQ8UD10I 2067186140 James S Sedlak Excellent price, quality and speed of delivery. A Good Deal 1.4634432E9 2016-05-17 1 3 null 0 1 1 3 5 7 47 1.0 67001081 4 true A2HGDUYV3B33Z1 4889960732 Smartens Good content. Four Stars 1.4078016E9 2014-08-12 2 2 2014-08-12 0 1 2 3 8 2 13 0.7 67001321 4 true A1ORWN1WRUYY5K 9058773221 Arlene Buchanan This is a small book, only 32 pages. The only reason I didn't give it five stars is because there aren't enough patterns. It does have instructions for beginners and examples of completed projects. It has the most elegant rose pattern which can be used in many color variations. Some reviewers have asked what papers to use for iris folding. Gift wrapping paper (new or used), origami paper, inside envelopes, tissue paper (crumple tissue for soft look) and ribbon. Our volunteer craft group has tried all these methods and you can't miss. Iris folding 1.2029472E9 2008-02-14 2 2 2007-10-23 114 1 2 5 2 97 544 0.20863636363636365 67001501 5 true A3N1ODG2O2VTRG 1503355845 Vivian Hwang What a beautiful story. The author wrote this as if she were actually in the action, in Kabul, or there, with Megan in her home. Very touching also. I definitely recommend this short story. Beautiful story 1.4187744E9 2014-12-17 1 5 null 0 1 1 4 12 34 189 0.26666666666666666 67001812 4 false A2NYQCLFQRTI74 9774241134 Francesca Jourdan This book relates many interesting stories about the Ancient Egyptian gods. Amongst those featured are the Daily Voyage of Ra through the sky, the mythical story of Osiris and Isis, the battle of Horus with his uncle Seth, the destruction of mankind by Hathor, the secret name of Ra, the book of Toth and the famous Tale of Two Brothers. There is a useful glossary of Ancient Egyptian gods. The book is clearly written with concise and brief explanations. good general reference to myths & legends of Ancient Egypt 9.608544E8 2000-06-13 2 1 2000-05-04 40 0 0 3 6 83 459 0.1777777777777778 67001989 3 true A1A09D62CJB11X B006U3QHYO Curt The text info appears to be good, and the numbering system is consistent with the dive map that I have. Apparently the paper copy has lots of photos in it, so I was disappointed to find that the kindle version had none. It really needs to incorporate a map at the least. Pretty good book, but the kindle version is devoid of photos and maps 1.377216E9 2013-08-23 1 2 null 0 1 1 6 8 54 272 0.025000000000000005 67002305 5 false A1C3TS5IO6E2NV 1499711492 Dr. Joyce M. Knudsen, Ph.D., AICI CIM Samantha Wright got me to do something I do not usually do...read a book in one evening! Through all of her experiences, trials and tribulations, she remained a woman of character after taking a step back every time she found herself in a difficult situation and demonstrated strength rarely seen in any person I've even known. No matter what happens in your life, within us, seems to be some mechanism that gets us through the tough times. If you have been through a horrific situation as written in NOT YOUR LEGACY, please read this book. It'll show you how religious beliefs can interfere with human relationships, how one can cope and try and understand why people do the things they do, but most of all, you'll discover the strength of a woman far beyond her years and know you will have the strength you need during difficult times. A great read; a wonderful woman of strength. Strength of Character 1.40408

In [31]:
print((test_df2.count(), len(test_df2.columns)))

In [32]:
#finalizing df for model
test_df2 = test_df2.drop('vote', 'unixReviewTime', 'Review_DoW', 'Review_Month', 'Review_Date')

#dropping sentiment for NB classifier (doesn't accept negative feature values)
#test_df2 = test_df2.drop('vote', 'unixReviewTime', 'Review_DoW', 'Review_Month', 'Review_Date', 'Sentiment')

#checking
print((test_df2.count(), len(test_df2.columns)))

(304984, 17)

In [33]:
display(test_df2)

In [34]:
kaggle_pred = pipelineFit.transform(test_df2)
kaggle_pred.show(5)

+--------+-------+--------+--------------+----------+-----------------+--------------------+---------------+-----------------+-------------------------+----------+----------------------+-------+--------------+----------------+-------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
reviewID|overall|verified| reviewerID| asin| reviewerName| reviewText| summary|review_number_new|product_review_number_new| prev_date|days_since_last_review|v_value|count_verified|Review_Wordcount|Review_Length| Sentiment| words| filtered| rawFeatures| idfFeatures| features| rawPrediction| probability|prediction|
+--------+-------+--------+--------------+----------+-----------------+--------------------+---------------+-----------------+-------------------------+----------+----------------------+-------+--------------+----------------+-------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
67000781| 5| false| AWG41315YSXQM|B00YTNMHEQ|Christine Frances|This book was exc...| Wow, wow, wow!| 1| 1| null| 0| 0| 0| 34| 168| 0.2688888888888889|[this, book, was,...|[book, exciting, ...|(2000,[1,5,27,73,...|(2000,[1,5,27,73,...|(2009,[1,2,3,6,7,...|[-0.6308459810756...|[0.22068276786780...| 1.0|
67001078| 5| true|A2FXMDRQ8UD10I|2067186140| James S Sedlak|Excellent price, ...| A Good Deal| 1| 3| null| 0| 1| 1| 7| 47| 1.0|[excellent, price...|[excellent, price...|(2000,[91,98,181,...|(2000,[91,98,181,...|(2009,[0,1,2,3,5,...|[0.92305830299677...|[0.86367050297757...| 0.0|
67001081| 4| true|A2HGDUYV3B33Z1|4889960732| Smartens| Good content.| Four Stars| 1| 2| null| 0| 1| 1| 2| 13| 0.7| [good, content]| [good, content]|(2000,[5,825],[1....|(2000,[5,825],[1....|(2009,[0,1,2,3,5,...|[0.51594771490133...|[0.73728319938575...| 0.0|
67001321| 4| true|A1ORWN1WRUYY5K|9058773221| Arlene Buchanan|This is a small b...| Iris folding| 2| 2|2007-10-23| 114| 1| 2| 97| 544|0.20863636363636365|[this, is, a, sma...|[small, book, 32,...|(2000,[1,12,21,22...|(2000,[1,12,21,22...|(2009,[0,1,2,3,4,...|[-0.8012374468435...|[0.16763599851264...| 1.0|
67001501| 5| true|A3N1ODG2O2VTRG|1503355845| Vivian Hwang|What a beautiful ...|Beautiful story| 1| 5| null| 0| 1| 1| 34| 189|0.26666666666666666|[what, a, beautif...|[beautiful, story...|(2000,[13,14,82,1...|(2000,[13,14,82,1...|(2009,[0,1,2,3,5,...|[-0.1116566791662...|[0.44440251718125...| 1.0|
+--------+-------+--------+--------------+----------+-----------------+--------------------+---------------+-----------------+-------------------------+----------+----------------------+-------+--------------+----------------+-------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
only showing top 5 rows

In [35]:
kaggle_pred.printSchema()

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-3883965750814009> in <module> () 
 ----> 1 kaggle_pred . printSchema ( ) 

 NameError : name 'kaggle_pred' is not defined

In [36]:
kaggle_pred.groupBy("prediction").count().show()

+----------+------+
prediction| count|
+----------+------+
 0.0| 50093|
 1.0|254891|
+----------+------+

In [37]:
display(kaggle_pred.select(["reviewID", "prediction"]))
#export to csv for submission!

reviewID,prediction
67000781,1.0
67001078,0.0
67001081,0.0
67001321,1.0
67001501,1.0
67001812,1.0
67001989,1.0
67002305,1.0
67002390,1.0
67002422,1.0
